In [1]:
# Author: Claire Wagner
# Date: 15 June 2022
# Purpose: To generate data about city-owned PINs.

In [2]:
import pandas as pd
import urllib.parse
import datetime

In [3]:
def makeAPIRequest(api_endpoint, params, limit, read_function):
    """Helper function to make Socrata API request."""
    query = "?" + "&".join(params) + "&$limit=" + str(limit)
    return read_function(api_endpoint + urllib.parse.quote(query, safe="&?$=,!()"))

def getLastUpdated(api_endpoint, read_function):
    return makeAPIRequest(
        api_endpoint = api_endpoint,
        params = [ "$select=max(:updated_at) AS last_updated" ],
        limit = 1,
        read_function = pd.read_json,
    ).loc[0, 'last_updated']

In [4]:
limit = 3000000
city_owned_api_endpoint = "https://data.cityofchicago.org/resource/aksk-kvfp.json"
properties_api_endpoint = "https://datacatalog.cookcountyil.gov/resource/c49d-89sn.json"

In [5]:
# fetch data from the City-Owned Land Inventory dataset about all properties currently owned by the City of Chicago that might be up for sale (see http://dev.cityofchicago.org/open%20data/data%20portal/2020/08/11/city-owned-property.html)
city_owned = makeAPIRequest(
    api_endpoint = city_owned_api_endpoint,
    params = [
        "$select=pin, managing_organization, lower(property_status) AS property_status, date_of_acquisition, date_of_disposition, sq_ft, last_update, :created_at, :updated_at",
        "$where=(lower(property_status)='owned by city') AND (lower(managing_organization)='none' OR managing_organization IS NULL)",
    ],
    limit = limit,
    read_function = pd.read_json,
)

In [6]:
#datetime.datetime.fromisoformat(city_owned_last_updated.split('T')[0]).strftime("%d %B %Y")

In [7]:
# fetch data from the Property Locations dataset about all properties in Wards 1-50
properties = makeAPIRequest(
    api_endpoint = properties_api_endpoint,
    params = [
        "$select=pin, property_address, property_zip, ward, longitude, latitude, tract_geoid, tract_pop, tract_midincome, tract_white_perc, tract_black_perc, tract_asian_perc, tract_his_perc, tract_other_perc",
    ],
    limit = limit,
    read_function = pd.read_json,
)

In [8]:
# Sunshine Gospel Ministries address (source: https://www.sunshinegospel.org/)
sgmAddress = "500 E 61st St".lower() # source: https://www.sunshinegospel.org/
# get location data for Sunshine Gospel Ministries
sgm = makeAPIRequest(
    api_endpoint = "https://datacatalog.cookcountyil.gov/resource/c49d-89sn.json",
    params = [
        "$select=pin, property_address, property_zip, ward, longitude, latitude, :created_at, :updated_at",
        f"$where=lower(property_address)='{sgmAddress}'",
    ],
    limit = 1,
    read_function = pd.read_json,
).loc[0]

In [9]:
# when datasets were last updated
city_owned_last_updated = getLastUpdated(
    api_endpoint = city_owned_api_endpoint,
    read_function = pd.read_json,
)
properties_last_updated = getLastUpdated(
    api_endpoint = properties_api_endpoint,
    read_function = pd.read_json,
)

In [10]:
# Generate attribution string that gives the URLs and the access date and time for the data sources.
fetchtime = datetime.datetime.now(datetime.timezone.utc).strftime("%d %B %Y at roughly %H:%M UTC")

In [11]:
attribution = f"/* Data sources: https://data.cityofchicago.org/Community-Economic-Development/City-Owned-Land-Inventory/aksk-kvfp (last updated at {city_owned_last_updated}) and https://datacatalog.cookcountyil.gov/Property-Taxation/Assessor-Archived-05-11-2022-Property-Locations/c49d-89sn (last updated at {properties_last_updated}), accessed on {fetchtime}. */"

In [12]:
# prepare to compute the inner join of these two datasets by standardizing the PIN format (convert to string, remove '-', left-pad with zeros)
city_owned["pin"] = city_owned["pin"].str.replace("-","") 
properties["pin"] = properties["pin"].apply(str).str.rjust(14, '0')

In [13]:
# compute the inner join
join = pd.merge(city_owned, properties, how="inner", on="pin", suffixes = ["_aksk-kvfp", "_c49d-89sn"])

In [14]:
# number of city-owned properties for which no location data from properties could be found
city_owned.shape[0] - join.shape[0]

6

In [15]:
join.filter(items=[
    "pin",
    "managing_organization",
    "property_status",
    "last_update",
    "date_of_acquisition",
    "date_of_disposition",
    "property_address",
    "property_zip",
    "ward",
    "tract_geoid",
]).to_excel("city_owned_pins_possibly_for_sale.xlsx", index=False)

In [16]:
join = join.set_index("pin", drop=False)

In [17]:
# output data to "data.js"
with open("../data/data.js", "w", encoding="utf-8") as f:
    f.write(attribution) # include attribution
    f.write(f"\n\nconst accessDateTime = \"{fetchtime}\";")
    f.write("\n\nconst sunshineGospel = ") # assign sgm JSON data to variable for easier access by JavaScript scripts in browser
    f.write(sgm.to_json(orient="index") + ";") # output sgm as JSON
    f.write("\n\nconst data = ") # assign join JSON data to variable for easier access by JavaScript scripts in browser
    f.write(join.to_json(orient="index") + ";") # output join as JSON

In [18]:
# scavenger sale data from 28 January 2022 (source: https://web.archive.org/web/20220128184252/https://www.cookcountytreasurer.com/pdfs/scavsale/2022cookcountyscavengertaxsalelist.xlsx, accessed 3 June 2022)
scavenger = pd.read_excel("../data/scavenger-sale/2022cookcountyscavengertaxsalelist_1-28-22.xlsx")

In [19]:
# standardize columns and PIN and classification formats
def standardize_columns(df):
    """standardize the names of all columns in df by making them lowercase and snake_case"""
    df.columns = df.columns.str.lower().str.split().str.join('_')
standardize_columns(scavenger)
scavenger["pin"] = scavenger["pin"].str.replace("-", "")
scavenger["classification"] = scavenger["classification"].str.strip()

In [38]:
# property class data (source: https://www.cookcountyassessor.com/form-document/codes-classification-property, accessed 21 June 2022)
property_classes = pd.read_excel("../data/scavenger-sale/property_classes.xlsx")

In [40]:
# join scavenger sale data with property class data
scavenger_with_property_classes = pd.merge(scavenger, property_classes, how="left", left_on = "classification", right_on="property_code").drop(columns=['classification'])

In [41]:
# make sure all scavenger sale entries have associated property class data
assert scavenger_with_property_classes[pd.isna(scavenger_with_property_classes['property_code'])].shape[0] == 0

In [42]:
# get location data for pins on scavenger sale list
scavenger_final_join = pd.merge(scavenger_with_property_classes, properties, how="inner", on="pin", suffixes=["_scavenger_sale", "_c49d-89sn"])

In [43]:
# make sure all pins on scavenger sale list have associated location data
assert scavenger_final_join.shape[0] == scavenger_with_property_classes.shape[0]

In [44]:
scavenger_final_join = scavenger_final_join[scavenger_final_join['property_city'] == 'CHICAGO'].filter(items = [
    'pin',
    'property_address_scavenger',
    'ward',
    'delinquent_tax_year_range',
    'delinquent_tax',
    'delinquent_interest',
    'total_delinquency',
    '2020_taxes_billed',
    'property_code',
    'property_code_meaning',
    'property_code_class',
    'property_zip',
    'longitude',
    'latitude',
    'tract_geoid',
    'tract_pop',
    'tract_midincome',
    'tract_white_perc',
    'tract_black_perc',
    'tract_asian_perc',
    'tract_his_perc',
    'tract_other_perc',
]).rename(columns = {
    'property_address_scavenger' : 'property_address',
}).set_index('pin', drop=False)

In [45]:
scavenger_data_attribution = f"/* PINs on the scavenger tax sale list where 'property_city' == 'CHICAGO'. Data sources: https://web.archive.org/web/20220128184252/https://www.cookcountytreasurer.com/pdfs/scavsale/2022cookcountyscavengertaxsalelist.xlsx (archived on 28 January 2022, accessed on 3 June 2022), https://datacatalog.cookcountyil.gov/Property-Taxation/Assessor-Archived-05-11-2022-Property-Locations/c49d-89sn (last updated at {properties_last_updated}, accessed on {fetchtime}), and https://www.cookcountyassessor.com/form-document/codes-classification-property (accessed 22 June 2022). */"

In [86]:
# make sure there are no duplicated entries for pins
assert scavenger_final_join.index.unique().shape[0] == scavenger_final_join.index.shape[0]

In [46]:
# output data to "scavenger_data.js"
with open("../data/scavenger-sale/scavenger_data.js", "w", encoding="utf-8") as f:
    f.write(scavenger_data_attribution) # include attribution
    f.write("\n\nconst scavenger_sale_data = ") # assign join JSON data to variable for easier access by JavaScript scripts in browser
    f.write(scavenger_final_join.to_json(orient="index") + ";") # output join as JSON

In [62]:
# Ward 20 Properties on Tax Sale list for Tax Year 2019, accessed on 17 May 2022 around 11 am ET
taxsale = pd.read_excel("../data/tax-sale/tax_sale_tax_year_2019_ward20_accessed_17_May_2022.xlsx")

In [71]:
standardize_columns(taxsale)
taxsale['pin'] = taxsale['pin'].str.replace('-', '')
taxsale = taxsale.filter(items=[
    'pin',
    'taxpayer_name',
    'property_address',
    'current_mailing_address',
    'tax_type',
    'tax_year',
    'total_tax_due',
    'total_due_(including_interest)',
    'classification',
    'prior_tax_years_may_also_be_unpaid',
])

In [100]:
# check how many entries have duplicate pins
taxsale['pin'].shape[0] - taxsale['pin'].unique().shape[0]

14

In [271]:
taxsale_to_aggregate = taxsale.set_index('pin')

In [272]:
duplicated = taxsale_to_aggregate.index[taxsale_to_aggregate.index.duplicated()]

In [273]:
concatCols = ['tax_type', 'tax_year']
sumCols = ['total_tax_due', 'total_due_(including_interest)']

for i in duplicated:
    for col in concatCols:
        concat = ''
        for count in range(len(taxsale_to_aggregate.loc[i, col])):
            element = str(taxsale_to_aggregate.loc[i, col][count])
            concat += element
            if count < len(taxsale_to_aggregate.loc[i, col]) - 1:
                concat += ', '
        taxsale_to_aggregate.loc[i, col] = concat

    for col in sumCols:
        taxsale_to_aggregate.loc[i, col] = taxsale_to_aggregate.loc[i, col].sum()
    tmp = taxsale_to_aggregate.loc[i].drop_duplicates()
    #taxsale_to_aggregate = taxsale_to_aggregate.drop([i])
    #taxsale_to_aggregate = pd.concat(objs=[taxsale_to_aggregate, tmp])

In [275]:
taxsale_to_aggregate.loc[['20161010770000', '20161010780000']].drop_duplicates()

,taxpayer_name,property_address,current_mailing_address,tax_type,tax_year,total_tax_due,total_due_(including_interest),classification,prior_tax_years_may_also_be_unpaid
pin,,,,,,,,,
20161010770000,LONNIE D. NEWMAN III,717 W GARFIELD BLVD,717 W GARFIELD BLVD CHICAGO IL 606212941,"1, 1, 1, 1, 1, 1, 1, 1, 1","2016, 2017, 2018, 2016, 2017, 2018, 2016, 2017...",8727.81,11305.29,Vacant Land,NaN
20161010780000,LONNIE NEWMANN,717 W GARFIELD BLVD,717 W GARFIELD BLVD CHICAGO IL 606212941,"1, 1, 1, 1, 1, 1, 1, 1, 1","2018, 2016, 2017, 2018, 2016, 2017, 2018, 2016...",113995.71,146274.00,Commercial/Industrial,NaN


In [94]:
taxsale_join = pd.merge(taxsale, properties, how="inner", on="pin", suffixes=["_tax_sale", "_c49d-89sn"])

In [95]:
# make sure all pins have associated location data and are in Ward 20
assert taxsale_join.shape[0] == taxsale.shape[0]
assert taxsale_join.shape[0] == taxsale_join[taxsale_join['ward'] == 20].shape[0]

In [96]:
taxsale_join = taxsale_join.drop(columns=[
    'property_address_c49d-89sn',
]).rename(columns={
    'property_address_tax_sale' : 'property_address',
    'total_due_(including_interest)' : 'total_due_including_interest',
})

14

In [101]:
taxsale['tax_year'].value_counts(dropna=False)

2019    1394
2018      15
2016       3
2017       3
Name: tax_year, dtype: int64

In [99]:
taxsale_join[taxsale_join.duplicated(subset='pin', keep=False)]

,pin,taxpayer_name,property_address,current_mailing_address,tax_type,tax_year,total_tax_due,total_due_including_interest,classification,prior_tax_years_may_also_be_unpaid,...,latitude,tract_geoid,tract_pop,tract_midincome,tract_white_perc,tract_black_perc,tract_asian_perc,tract_his_perc,tract_other_perc,ward
97,20093290220000,CANDIS POWELL,609 W 54TH PL,609 W 54TH PL CHICAGO IL 606090000,0,2019,402.86,527.62,Vacant Land,NaN,...,41.795041,1.703184e+10,1482.0,34375.0,0.238866,0.566127,0.067476,0.123482,0.008097,20.0
98,20093290220000,CANDIS POWELL,609 W 54TH PL,609 W 54TH PL CHICAGO IL 606090000,1,2016,407.83,534.11,Vacant Land,NaN,...,41.795041,1.703184e+10,1482.0,34375.0,0.238866,0.566127,0.067476,0.123482,0.008097,20.0
144,20161010770000,LONNIE D. NEWMAN III,717 W GARFIELD BLVD,717 W GARFIELD BLVD CHICAGO IL 606212941,1,2016,984.80,1275.43,Vacant Land,NaN,...,41.793703,1.703183e+10,1426.0,22104.0,0.000701,0.884993,0.029453,0.077840,0.014025,20.0
145,20161010770000,LONNIE D. NEWMAN III,717 W GARFIELD BLVD,717 W GARFIELD BLVD CHICAGO IL 606212941,1,2017,965.07,1250.19,Vacant Land,NaN,...,41.793703,1.703183e+10,1426.0,22104.0,0.000701,0.884993,0.029453,0.077840,0.014025,20.0
146,20161010770000,LONNIE D. NEWMAN III,717 W GARFIELD BLVD,717 W GARFIELD BLVD CHICAGO IL 606212941,1,2018,959.40,1242.81,Vacant Land,NaN,...,41.793703,1.703183e+10,1426.0,22104.0,0.000701,0.884993,0.029453,0.077840,0.014025,20.0
147,20161010780000,LONNIE NEWMANN,717 W GARFIELD BLVD,717 W GARFIELD BLVD CHICAGO IL 606212941,1,2018,11855.55,15244.32,Commercial/Industrial,NaN,...,41.793706,1.703183e+10,1426.0,22104.0,0.000701,0.884993,0.029453,0.077840,0.014025,20.0
148,20161010780000,LONNIE NEWMANN,717 W GARFIELD BLVD,717 W GARFIELD BLVD CHICAGO IL 606212941,1,2016,13222.98,16901.44,Commercial/Industrial,NaN,...,41.793706,1.703183e+10,1426.0,22104.0,0.000701,0.884993,0.029453,0.077840,0.014025,20.0
149,20161010780000,LONNIE NEWMANN,717 W GARFIELD BLVD,717 W GARFIELD BLVD CHICAGO IL 606212941,1,2017,12920.04,16612.24,Commercial/Industrial,NaN,...,41.793706,1.703183e+10,1426.0,22104.0,0.000701,0.884993,0.029453,0.077840,0.014025,20.0
246,20161210050000,PIVOT URBAN,5811 S EMERALD AVE,325 W HURON STE 701 CHICAGO IL 606543617,0,2019,408.44,534.91,Vacant Land,NaN,...,41.788454,1.703183e+10,1426.0,22104.0,0.000701,0.884993,0.029453,0.077840,0.014025,20.0
247,20161210050000,PIVOT URBAN LLC,5811 S EMERALD AVE,325 W HURON STE 701 CHICAGO IL 606543617,1,2018,401.60,525.98,Vacant Land,NaN,...,41.788454,1.703183e+10,1426.0,22104.0,0.000701,0.884993,0.029453,0.077840,0.014025,20.0


In [69]:
taxsale_data_attribution = f"/* PINs in Ward 20 on the tax sale list for tax year 2019. Data sources: https://www.cookcountytreasurer.com/delinquenttaxes.aspx (accessed on 17 May 2022 around 11 am ET) and https://datacatalog.cookcountyil.gov/Property-Taxation/Assessor-Archived-05-11-2022-Property-Locations/c49d-89sn (last updated at {properties_last_updated}, accessed on {fetchtime})."

In [ ]:
# output data to "scavenger_data.js"
with open("../data/taxsale/taxsale_ward20.js", "w", encoding="utf-8") as f:
    f.write(taxsale_data_attribution) # include attribution
    f.write("\n\nconst taxsale_ward20_data = ") # assign join JSON data to variable for easier access by JavaScript scripts in browser
    f.write(scavenger_final_join.to_json(orient="index") + ";") # output join as JSON